# Converting STELLA Files to be usable by TARDIS

In [1]:
from tardis.io.parsers import stella
from astropy import units as u

/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/setuptools_scm/git.py:105: UserWarning: "/home/runner/work/tardis/tardis" is shallow and may cause errors
  warnings.warn(f'"{wd.path}" is shallow and may cause errors')


/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/traitlets/traitlets.py:3044: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
data = stella.read_stella_data('mesa.stella.dat')

In [3]:
data

,mass_of_cell,cell_center_m,cell_center_R,cell_center_v,avg_density,radiation_pressure,avg_temperature,radiation_temperature,avg_opacity,tau,...,cr48,cr60,fe52,fe54,fe56,co56,ni56,luminosity,n_bar,n_e
1,6.006769e+29,5.190243e+33,2.517209e+13,3.930633e+06,1.005529e-11,11237.954506,45943.453023,45943.453023,0.162712,5403.504881,...,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03,7.648868e+38,6.055413e+12,2.412188e+12
2,1.262406e+30,5.191506e+33,2.970626e+13,5.050403e+06,2.928088e-11,11236.544022,45942.182229,45942.182229,0.164412,5381.678444,...,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03,9.609214e+38,1.762304e+13,6.969790e+12
3,1.264624e+30,5.192771e+33,3.198879e+13,5.680288e+06,4.619659e-11,11133.526767,45836.518142,45836.518142,0.165012,5364.280084,...,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03,1.050921e+39,2.783916e+13,1.091728e+13
4,1.263067e+30,5.194034e+33,3.357130e+13,6.135230e+06,5.896887e-11,11133.526767,45836.518142,45836.518142,0.165310,5348.854704,...,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03,1.100921e+39,3.545183e+13,1.392188e+13
5,1.259387e+30,5.195293e+33,3.480638e+13,6.495173e+06,6.912717e-11,11032.838720,45732.362031,45732.362031,0.165510,5334.724893,...,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03,1.140921e+39,4.166219e+13,1.632649e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1.511505e+31,2.611308e+34,1.499835e+15,3.349250e+08,1.908195e-14,0.143918,3319.925147,2748.482551,0.000405,0.001398,...,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14,3.729654e+41,1.151612e+10,5.573015e+05
396,1.511731e+31,2.612818e+34,1.533756e+15,3.435050e+08,1.540408e-14,0.136196,3259.329684,2710.845252,0.000430,0.001173,...,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14,3.729654e+41,9.283015e+09,4.408753e+05
397,1.511951e+31,2.614332e+34,1.576303e+15,3.545750e+08,1.168519e-14,0.128874,3192.480784,2673.651060,0.000467,0.000941,...,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14,3.719770e+41,7.040251e+09,3.305759e+05
398,1.512167e+31,2.615844e+34,1.632356e+15,3.682800e+08,8.348776e-15,0.119719,3126.918024,2624.851429,0.000525,0.000695,...,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14,3.719654e+41,5.027256e+09,2.333801e+05


In [4]:
# extract outer radius boundaries 
radii = data.loc[:,'outer_edge_r'].values * u.cm

# calculate t_explosion
t_explosion = 50 * u.day #days

# calculate outer velocities of each zone assuming homologous expansion (v=r/t) 
velocities = (radii/t_explosion).to(u.km/u.s) 

# create a new column of our velocities (OPTIONAL)
data['velocity'] = velocities

In [5]:
density_dat = data[['velocity', 'avg_density']].reset_index(drop=True)
density_dat.to_csv('density_parse.csv', sep = ' ')

In [6]:
# import all elements and isotopes and export to TARDIS
data_elements = data.iloc[:,12:33].reset_index(drop=True)
data_elements.to_csv('abund_parse.csv', sep = ' ')

In [7]:
density_dat

,velocity,avg_density
0,63.516608,1.005529e-11
1,71.406308,2.928088e-11
2,75.879732,4.619659e-11
3,79.140834,5.896887e-11
4,81.760282,6.912717e-11
...,...,...
394,3511.100633,1.908195e-14
395,3599.605377,1.540408e-14
396,3713.726589,1.168519e-14
397,3854.655008,8.348776e-15


In [8]:
data_elements

,h1,he3,he4,c12,n14,o16,ne20,na23,mg24,si28,...,ar36,ca40,ti44,cr48,cr60,fe52,fe54,fe56,co56,ni56
0,0.000035,0.0,0.580099,0.119035,7.025597e-07,0.246159,0.006333,3.274956e-08,0.007389,0.031126,...,0.000483,0.000170,0.0,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03
1,0.000035,0.0,0.580099,0.119035,7.025597e-07,0.246159,0.006333,3.274956e-08,0.007389,0.031126,...,0.000483,0.000170,0.0,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03
2,0.000035,0.0,0.580099,0.119035,7.025597e-07,0.246159,0.006333,3.274956e-08,0.007389,0.031126,...,0.000483,0.000170,0.0,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03
3,0.000035,0.0,0.580099,0.119035,7.025597e-07,0.246159,0.006333,3.274956e-08,0.007389,0.031126,...,0.000483,0.000170,0.0,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03
4,0.000035,0.0,0.580099,0.119035,7.025597e-07,0.246159,0.006333,3.274956e-08,0.007389,0.031126,...,0.000483,0.000170,0.0,0.0,0.0,0.0,0.0,0.000021,0.0,4.079500e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0.718125,0.0,0.276054,0.000640,1.019424e-03,0.002458,0.000436,4.752008e-05,0.000849,0.000143,...,0.000019,0.000015,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14
395,0.718125,0.0,0.276054,0.000640,1.019424e-03,0.002458,0.000436,4.752008e-05,0.000849,0.000143,...,0.000019,0.000015,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14
396,0.718125,0.0,0.276054,0.000640,1.019424e-03,0.002458,0.000436,4.752008e-05,0.000849,0.000143,...,0.000019,0.000015,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14
397,0.718125,0.0,0.276054,0.000640,1.019424e-03,0.002458,0.000436,4.752008e-05,0.000849,0.000143,...,0.000019,0.000015,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,1.108306e-14
